In [6]:
import numpy as np

def softmax(x):
    b = np.max(x)
    numerator = np.exp(x-b)
    probs = numerator/np.sum(numerator,axis=1,keepdims=True)
    return probs #Normalized probabilities of each class

class NeuralNet:
    
    def __init__(self,dimensions):
        self.weights = [0]*(len(dimensions)-1)
        self.biases = [0]*(len(dimensions)-1)
        for i in range(len(dimensions)-1):
            self.weights[i] = np.random.uniform(low=(-1)/float((np.sqrt(dimensions[i]))),
                                                high=(1)/float((np.sqrt(dimensions[i]))),
                                                size=((dimensions[i],dimensions[i+1]))
                                               )
            self.biases[i] = np.zeros((dimensions[i+1]))   
        self.D = dimensions[len(dimensions)-1]
        
    def loss(self,probs,labels):
        N = probs.shape[0]
        loss = -np.sum((np.log(probs[np.arange(N),labels])))
        loss /= N
        dprobs = np.zeros_like(probs)
        return loss
    
    def fprop(self,inp,labels):
        N = inp.shape[0]
        #determine product of all dimensions
        D = np.prod(inp.shape[1:])
        #reshape inputs to Number of examples x product of all dimensions
        data = inp.reshape((N,D))
        activations1 = np.dot(data,self.weights[0]) + self.biases[0]
        hidden1 = np.maximum(0,activations1)
        
        activations2 = np.dot(hidden1,self.weights[1]) + self.biases[1]
        
        probs = softmax(activations2)
        
        loss = self.loss(probs,labels)
        cache = (inp,activations1,hidden1,activations2,probs,labels)
        return loss, cache

    def bprop(self,cache):
        inp, activations1, hidden1, activations2, probs,labels = cache
        N = inp.shape[0]
        #grads of softmax function
        grad_oa = probs
        grad_oa[np.arange(labels.shape[0]),labels] -= 1
        #grads of layer 2
        grad_W2 = np.dot(np.transpose(hidden1),grad_oa)/N
        grad_b2 = np.sum(grad_oa,axis=0)/N
        grad_hs = np.dot(grad_oa,np.transpose(self.weights[1]))

        #Gradient through Relu nonlinearity
        grad_ha = grad_hs*(np.where(activations1>0,1,0))

        #grads of input layer
        grad_W1 = np.dot(np.transpose(inp),grad_ha)/N
        grad_b1 = np.sum(grad_ha,axis=0)/N
        grad_inp = np.dot(grad_ha,np.transpose(self.weights[0]))/N

        return (grad_inp, grad_W1, grad_b1, grad_ha, grad_hs,grad_W2, grad_b2, grad_oa)
        
    def grad_check(self,inp,labels,epsilon):
        
        actual_loss, cache = self.fprop(inp,labels)
        
        (_,grad_W1, grad_b1, _,_,grad_W2,grad_b2,_) = self.bprop(cache)
        #Gradient check on b2
        for i in range(self.biases[1].shape[0]):
            self.biases[1][i] += epsilon
            loss_perturbed_b2,_ = self.fprop(inp,labels)
            self.biases[1][i] -= epsilon
            check_grad = (loss_perturbed_b2-actual_loss)/epsilon
            if grad_b2[i] == 0:
                print (check_grad+epsilon)/(grad_b2[i]+epsilon)
            else:
                print check_grad/grad_b2[i]
        #Gradient check on W2 weights
        for i in range(self.weights[1].shape[0]):
            for j in range(self.weights[1].shape[1]):
                self.weights[1][i,j] += epsilon
                loss_perturbed_W2,_ = self.fprop(inp,labels)
                self.weights[1][i,j] -= epsilon
                check_grad = (loss_perturbed_W2-actual_loss)/epsilon
                if grad_W2[i,j] == 0:
                    print (check_grad+epsilon)/(grad_W2[i,j]+epsilon)
                else:
                    print check_grad/grad_W2[i,j]

        #Gradient check on b1
        for i in range(self.biases[0].shape[0]):
            self.biases[0][i] += epsilon
            loss_perturbed_b1,_ = self.fprop(inp,labels)
            self.biases[0][i] -= epsilon
            check_grad = (loss_perturbed_b1-actual_loss)/epsilon
            if grad_b1[i] == 0:
                print (check_grad+epsilon)/(grad_b1[i]+epsilon)
            else:
                print check_grad/grad_b1[i]
        #Gradient check on W2 weights
        for i in range(self.weights[0].shape[0]):
            for j in range(self.weights[0].shape[1]):
                self.weights[0][i,j] += epsilon
                loss_perturbed_W1,_ = self.fprop(inp,labels)
                self.weights[0][i,j] -= epsilon
                check_grad = (loss_perturbed_W1-actual_loss)/epsilon
                if grad_W1[i,j] == 0:
                    print (check_grad+epsilon)/(grad_W1[i,j]+epsilon)
                else:
                    print check_grad/grad_W1[i,j]
                    
    def train(self,data,learning_rate,batch_size,num_epochs):
        num_steps = int(float(data.shape[0]/float(batch_size)))
        for epoch in range(num_epochs):
            for step in range(num_steps):
                lower_bound = (step*batch_size)%data.shape[0]
                upper_bound =(((step+1)*batch_size)%data.shape[0])
                if upper_bound < lower_bound:
                    upper_bound = data.shape[0]
                features = data[lower_bound:upper_bound,:-1]
                labels = data[lower_bound:upper_bound,-1].astype(np.int)
                print features.shape
                print labels.shape
                loss, cache = self.fprop(features,labels)
                (grad_inp, grad_W1, grad_b1, grad_ha, grad_hs,grad_W2, grad_b2, grad_oa) = self.bprop(cache)

                #Apply gradient descent
                self.weights[0] -= learning_rate*grad_W1
                self.biases[0] -= learning_rate*grad_b1
                self.weights[1] -= learning_rate*grad_W2
                self.biases[1] -= learning_rate*grad_b2
                
                
            


np.random.seed(123)            
NN = NeuralNet([10,40,2])
example = np.random.uniform(size=(10,10))#[1,2,3,4,5,6,7,8,9,10]
labels = np.ones((10,),dtype=np.int)
loss,(inp,activations1,hidden1,activations2,probs,labels) = NN.fprop(example,labels)

grads = NN.bprop((inp,activations1,hidden1,activations2,probs,labels))
#NN.grad_check(example,labels,0.00001,grads)


In [240]:
def onehotify(labels,num_classes):
    labels = labels.astype(int)
    onehots = np.zeros((labels.shape[0],num_classes))
    onehots[np.arange(labels.shape[0]),labels] = 1
    return onehots

class NeuralNet_single:
    def __init__(self,dimensions):
        self.weights = [0]*(len(dimensions)-1)
        self.biases = [0]*(len(dimensions)-1)
        for i in range(len(dimensions)-1):
            self.weights[i] = np.random.uniform(low=(-1)/float((np.sqrt(dimensions[i]))),
                                                high=(1)/float((np.sqrt(dimensions[i]))),
                                                size=((dimensions[i+1],dimensions[i]))
                                               )
            self.biases[i] = np.zeros((dimensions[i+1]))   
        self.D = dimensions[len(dimensions)-1]
    def loss(self,logit,label):
        loss = -np.log(logit[np.argmax(label)])
        return loss
    
    def softmax(self,x):
        b = np.max(x)
        numerator = np.exp(x-b)
        return numerator/np.sum(numerator)
        
    def fprop(self,inp,labels,train=True):
        #determine product of all dimensions
        D = np.prod(inp.shape[:])
        #reshape inputs to Number of examples x product of all dimensions
        data = inp.reshape((D))
        activations1 = np.dot(self.weights[0],data) + self.biases[0]
        hidden1 = np.maximum(0,activations1)

        activations2 = np.dot(self.weights[1],hidden1) + self.biases[1]
        
        probs = self.softmax(activations2)
        if (not train):
            return probs

        loss = self.loss(probs,labels)
        cache = (inp,activations1,hidden1,activations2,probs,labels)
        
        return loss, cache
        
    def bprop(self,cache):
        inp, activation1, hidden1, activation2, probs,label = cache
        #grads of softmax function
        grad_oa = probs
        grad_oa[np.argmax(label)] -= 1
        #grads of layer 2
        grad_W2 = np.outer(grad_oa,hidden1)
        grad_b2 = grad_oa
        grad_hs = np.dot(np.transpose(self.weights[1]),grad_oa)

        #Gradient through Relu nonlinearity
        grad_ha = grad_hs*(np.where(activation1>0,1,0))

        #grads of input layer
        grad_W1 = np.outer(grad_ha,inp)
        grad_b1 = grad_ha
        grad_inp = np.dot(np.transpose(self.weights[0]),grad_ha)

        return (grad_inp, grad_W1, grad_b1, grad_ha, grad_hs,grad_W2, grad_b2, grad_oa)
        
    def train(self,data, learning_rate,num_classes, batch_size):
        features = data[:,:-1]
        labels = data[:,-1]
        onehotlabels = onehotify(labels,num_classes)
        
        for i in range(data.shape[0]/batch_size):
            loss = 0
            grad_inp = grad_W1 = grad_b1 = grad_ha = grad_hs = grad_W2 = grad_b2 = grad_oa = 0
            for j in range(batch_size):
                sample_loss, cache = self.fprop(features[(i*batch_size)+j,:],onehotlabels[(i*batch_size)+j,:])
                loss += sample_loss
                (sample_grad_inp, sample_grad_W1, sample_grad_b1, sample_grad_ha,
                 sample_grad_hs,sample_grad_W2, sample_grad_b2, sample_grad_oa) = self.bprop(cache)
                grad_W1 += sample_grad_W1
                grad_b1 += sample_grad_b1
                grad_W2 += sample_grad_W2
                grad_b2 += sample_grad_b2
                
            loss /= batch_size
            grad_W1 /= batch_size
            grad_b1 /= batch_size
            grad_W2 /= batch_size
            grad_b2 /= batch_size
            #gradient descent
            self.weights[0] -= learning_rate*grad_W1
            self.biases[0] -= learning_rate*grad_b1
            self.weights[1] -= learning_rate*grad_W2
            self.biases[1] -= learning_rate*grad_b2
                
            print 'Batch loss: ' + repr(loss)
            
    def grad_check(self,data,epsilon,num_classes, batch_size = 1):
        
        inp = data[:,:-1]
        labels = data[:,-1]
        batch_size = data.shape[0]
        onehotlabels = onehotify(labels,num_classes)
        actual_loss = grad_W1 = grad_b1 = grad_W2 = grad_b2 = 0
        for i in range(batch_size):
            sample_actual_loss,(cache) = self.fprop(inp[i,:],onehotlabels[i,:])
            (_,sample_grad_W1, sample_grad_b1, _,_,sample_grad_W2,sample_grad_b2,_) = self.bprop(cache)
            actual_loss += sample_actual_loss
            grad_W1 += sample_grad_W1
            grad_b1 += sample_grad_b1
            grad_W2 += sample_grad_W2
            grad_b2 += sample_grad_b2
        actual_loss /= batch_size
        grad_W1 /= batch_size
        grad_b1 /= batch_size
        grad_W2 /= batch_size
        grad_b2 /= batch_size

        
        #Gradient check on b2
        print 'Second Bias gradients'
        for i in range(self.biases[1].shape[0]):
            self.biases[1][i] += epsilon
            loss_perturbed_b2 = 0
            for j in range(batch_size):
                sample_loss_perturbed_b2,_ = self.fprop(inp[j,:],onehotlabels[j,:])
                loss_perturbed_b2 += sample_loss_perturbed_b2
            loss_perturbed_b2 /= batch_size
            self.biases[1][i] -= epsilon
            check_grad = (loss_perturbed_b2-actual_loss)/epsilon
            if grad_b2[i] == 0:
                print 'Ratio of grads: ' + repr((check_grad+epsilon)/(grad_b2[i]+epsilon))
            else:
                print 'Ratio of grads: ' + repr(check_grad/grad_b2[i])
            print 'Gradient b2[' + repr(i) + ']: Finite difference: ' + repr(check_grad) + ' Analytical: ' + repr(grad_b2[i])
                
        #Gradient check on W2 weights
        print 'Second weight gradients'
        for i in range(self.weights[1].shape[0]):
            for j in range(self.weights[1].shape[1]):
                self.weights[1][i,j] += epsilon
                loss_perturbed_W2 = 0
                for k in range(batch_size):
                    sample_loss_perturbed_W2,_ = self.fprop(inp[k,:],onehotlabels[k,:])
                    loss_perturbed_W2 += sample_loss_perturbed_W2
                loss_perturbed_W2 /= batch_size
                self.weights[1][i,j] -= epsilon
                check_grad = (loss_perturbed_W2-actual_loss)/epsilon
                if grad_W2[i,j] == 0:
                    print 'Ratio of grads: ' + repr((check_grad+epsilon)/(grad_W2[i,j]+epsilon))
                else:
                    print 'Ratio of grads: ' + repr(check_grad/grad_W2[i,j])
                print 'Gradient W2[' + repr(i) + ',' + repr(j) + ']: Finite difference: ' + repr(check_grad) + ' Analytical: ' + repr(grad_W2[i,j])

        #Gradient check on b1
        print 'First bias gradients'
        for i in range(self.biases[0].shape[0]):
            self.biases[0][i] += epsilon
            loss_perturbed_b1 = 0
            for j in range(batch_size):
                sample_loss_perturbed_b1,_ = self.fprop(inp[j,:],onehotlabels[j,:])
                loss_perturbed_b1 += sample_loss_perturbed_b1
            loss_perturbed_b1 /= batch_size
            self.biases[0][i] -= epsilon
            check_grad = (loss_perturbed_b1-actual_loss)/epsilon
            if grad_b1[i] == 0:
                print 'Ratio of grads: ' + repr((check_grad+epsilon)/(grad_b1[i]+epsilon))
            else:
                print 'Ratio of grads: ' + repr(check_grad/grad_b1[i])
            print 'Gradient b1[' + repr(i) + ']: Finite difference: ' + repr(check_grad) + ' Analytical: ' + repr(grad_b1[i])
        #Gradient check on W1 weights
        print 'First weights gradients'
        for i in range(self.weights[0].shape[0]):
            for j in range(self.weights[0].shape[1]):
                self.weights[0][i,j] += epsilon
                loss_perturbed_W1 = 0
                for k in range(batch_size):
                    sample_loss_perturbed_W1,_ = self.fprop(inp[k,:],onehotlabels[k,:])
                    loss_perturbed_W1 += sample_loss_perturbed_W1
                loss_perturbed_W1 /= batch_size
                self.weights[0][i,j] -= epsilon
                check_grad = (loss_perturbed_W1-actual_loss)/epsilon
                if grad_W1[i,j] == 0:
                    print 'Ratio of grads: ' + repr((check_grad+epsilon)/(grad_W1[i,j]+epsilon)
                else:
                    print check_grad/grad_W1[i,j]
                print 'Gradient W1[' + repr(i) + ',' + repr(j) + ']: Finite difference: ' + repr(check_grad) + ' Analytical: ' + repr(grad_W1[i,j])
                    

In [241]:
singleNN = NeuralNet_single([2,2,2])

features = np.random.uniform(size=(1,2))
labels = np.ones((1,1))

data = np.hstack((features,labels))

singleNN.grad_check(data,0.00001,2)
#singleNN.train(data,0.01,2,10)

Second Bias gradients
Gradient b2[0]: Finite difference: 0.49275242447466278 Analytical: 0.49275117472987151
Gradient b2[1]: Finite difference: -0.49274992500736298 Analytical: -0.49275117472987151
Second weight gradients
Gradient W2[0,0]: Finite difference: 0.0021699110730466487 Analytical: 0.002169911046677386
Gradient W2[0,1]: Finite difference: 0.052680107243130429 Analytical: 0.052680092964194396
Gradient W2[1,0]: Finite difference: -0.0021699110286377277 Analytical: -0.002169911046677386
Gradient W2[1,1]: Finite difference: -0.052680078677092006 Analytical: -0.052680092964194396
First bias gradients
Gradient b1[0]: Finite difference: 0.12114772032223441 Analytical: 0.12114764480279899
Gradient b1[1]: Finite difference: -0.1386393656876983 Analytical: -0.1386394646076497
First weights gradients
Gradient W1[0,0]: Finite difference: 0.01617119074204254 Analytical: 0.016171189412594041
Gradient W1[0,1]: Finite difference: 0.022164371216160102 Analytical: 0.0221643686841385
Gradient W

Data Set Up: 2 Moons

In [242]:
fname = "twomoons.txt"
data = np.loadtxt(open(fname,'r'))

Experiment 1: Gradient Check on a single example

In [243]:
singleNN = NeuralNet_single([2,2,2])
#take one example
example = data[0,:]
# reshape data from single column vectore to single row vector
example = np.reshape(example,(1,3))
singleNN.grad_check(example,0.00001,2)

Second Bias gradients
Gradient b2[0]: Finite difference: -0.43816959965914387 Analytical: -0.43817083053620498
Gradient b2[1]: Finite difference: 0.43817206141216752 Analytical: 0.43817083053620498
Second weight gradients
Gradient W2[0,0]: Finite difference: -0.25171286168745866 Analytical: -0.25171326787874193
Gradient W2[0,1]: Finite difference: 0.0 Analytical: 0.0
Gradient W2[1,0]: Finite difference: 0.25171367407095246 Analytical: 0.25171326787874193
Gradient W2[1,1]: Finite difference: 0.0 Analytical: 0.0
First bias gradients
Gradient b1[0]: Finite difference: -0.18961040055831543 Analytical: -0.18961063105792506
Gradient b1[1]: Finite difference: 0.0 Analytical: 0.0
First weights gradients
Gradient W1[0,0]: Finite difference: -0.069285235348992558 Analytical: -0.06928526612299632
Gradient W1[0,1]: Finite difference: 0.340489047212511 Analytical: 0.3404883039626721
Gradient W1[1,0]: Finite difference: 0.0 Analytical: 0.0
Gradient W1[1,1]: Finite difference: 0.0 Analytical: 0.0
